In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from glob import glob
from datetime import datetime
import sys
sys.path.append('/Users/w76m851/github_workspace/spicy-snow/')

from spicy_snow.processing.snow_index import calc_delta_cross_ratio, calc_delta_gamma, \
    clip_delta_gamma_outlier, calc_snow_index, calc_snow_index_to_snow_depth
from spicy_snow.processing.wet_snow import id_newly_wet_snow, id_wet_negative_si, \
    id_newly_frozen_snow, flag_wet_snow


In [2]:
files = sorted(glob('/Users/w76m851/OneDrive - Montana State University/spicy_s1_stacks/*.nc'))
test = xr.open_dataset(files[0])
test

<xarray.Dataset>
Dimensions:         (x: 185, y: 144, band: 3, time: 58)
Coordinates:
  * x               (x) float64 -115.3 -115.3 -115.3 ... -115.1 -115.1 -115.1
  * y               (y) float64 44.34 44.34 44.34 44.34 ... 44.2 44.19 44.19
  * band            (band) object 'VV' 'VH' 'inc'
  * time            (time) datetime64[ns] 2019-08-05T01:35:54 ... 2020-03-02T...
    flight_dir      (time) object ...
    platform        (time) object ...
    relative_orbit  (time) float64 ...
    projection      int64 ...
Data variables: (12/17)
    s1              (time, band, y, x) float32 ...
    spatial_ref     int64 ...
    ims             (time, y, x) float64 ...
    fcf             (y, x) float64 ...
    deltaCR         (time, y, x) float32 ...
    deltaVV         (time, y, x) float32 ...
    ...              ...
    freeze_flag     (time, y, x) float32 ...
    wet_snow        (time, y, x) float32 ...
    perma_wet       (time, y, x) float32 ...
    lidar-sd        (y, x) float32 ...
    lidar-vh        (y, x) float32 ...
    lidar-dem       (y, x) float32 ...
Attributes:
    AREA_OR_POINT:      Point
    TIFFTAG_DATETIME:   2023:02:25 03:57:04
    TIFFTAG_SOFTWARE:   Created with GAMMA Software www.gamma-rs.ch data2geot...
    scale_factor:       1.0
    add_offset:         0.0
    resolution:         90
    s1_units:           dB
    site:               Banner
    site_abbrev:        USIDBS
    lidar-flight-time:  2020-02-18

In [40]:
# %%time
for f in files[-2]:
    ds_name = f.split('stacks/')[-1].split('.')[0]
    if 'Little' in ds_name:
        continue
    print(datetime.now(), f' -- starting {ds_name}')
    # Open dataset 
    ds_ = xr.open_dataset(f)
    dataset = ds_[['s1','deltaVV','ims','fcf','lidar-sd']]
    td = abs(pd.to_datetime(dataset.time) - pd.to_datetime(dataset.attrs['lidar-flight-time']))
    closest_ts_idx = np.where(td == td.min())[0][0]
    closest_ts = dataset.time[closest_ts_idx]
    # Initialize RMSE arrays
    rmse_wet_flag = xr.DataArray(np.empty((len(A), len(B), len(C)))*np.nan,
                        coords=(A, B, C), dims=('A','B','C'))
    rmse_no_flag = xr.DataArray(np.empty((len(A), len(B), len(C)))*np.nan,
                        coords=(A, B, C), dims=('A','B','C'))
    # Create parameter space
    A = np.arange(1, 3.1, 1)
    B = np.arange(0, 1.01, 0.2)
    C = np.arange(0, 1.001, 0.05)
    # Brute-force loop
    for a in A:
        ds = calc_delta_cross_ratio(dataset, A=a, inplace=False)
        for b in B:
            ds = calc_delta_gamma(ds, B=b, inplace=False)
            for c in C:
                # print(f'A={a}; B={b}; C={c}')
                ds = clip_delta_gamma_outlier(ds)
                ds = calc_snow_index(ds)
                ds = calc_snow_index_to_snow_depth(ds, C=c, inplace=False)
                ds = id_newly_wet_snow(ds)
                ds = id_wet_negative_si(ds)
                ds = id_newly_frozen_snow(ds)
                ds = flag_wet_snow(ds)
                # Compare snow depths - mask wet snow
                mask_wet = ~(pd.isnull(ds['lidar-sd']) | pd.isnull(ds['snow_depth'].sel(time=closest_ts)) | 
                        ds['wet_snow'].sel(time=closest_ts).astype(bool))
                diff_wet = ds['lidar-sd'].where(mask_wet) - ds['snow_depth'].sel(time=closest_ts).where(mask_wet)
                rmse_wet = float(np.sqrt((diff_wet**2).sum()/len(diff_wet.values.flatten())))
                rmse_wet_flag.loc[a, b, c] = rmse_wet
                # Compare snow depths - no wet snow mask
                mask = ~(pd.isnull(ds['lidar-sd']) | pd.isnull(ds['snow_depth'].sel(time=closest_ts)))
                diff = ds['lidar-sd'].where(mask) - ds['snow_depth'].sel(time=closest_ts).where(mask)
                rmse = float(np.sqrt((diff**2).sum()/len(diff.values.flatten())))
                rmse_no_flag.loc[a,b,c] = rmse

    # After loop, save RMSE results per file
    rmse_wet_flag.to_netcdf(f'rmse_out/{ds_name}_wet_flag.nc')
    rmse_no_flag.to_netcdf(f'rmse_out/{ds_name}_no_flag.nc')
    

2023-03-02 21:29:09.879256  -- starting Banner_2020-02-18
2023-03-02 22:02:06.700167  -- starting Banner_2021-03-15
2023-03-02 22:56:34.138678  -- starting Cameron_2021-03-19
2023-03-02 23:26:27.596496  -- starting Dry_Creek_2020-02-19
2023-03-03 00:07:08.752166  -- starting Frasier_2020-02-11
2023-03-03 00:31:43.291953  -- starting Frasier_2021-03-19
2023-03-03 01:01:15.682588  -- starting Little_Cottonwood_2021-03-18


IndexError: The indexing operation you are attempting to perform is not valid on netCDF4.Variable object. Try loading your data into memory first by calling .load().

In [141]:
for f in files[5:6]:
    ds_name = f.split('stacks/')[-1].split('.')[0]
    if 'Little' in ds_name:
        continue
    print(datetime.now(), f' -- starting {ds_name}')
    # Open dataset 
    ds_ = xr.open_dataset(f)
    dataset = ds_[['s1','deltaVV','ims','fcf','lidar-sd']]
    td = abs(pd.to_datetime(dataset.time) - pd.to_datetime(dataset.attrs['lidar-flight-time']))
    closest_ts_idx = np.where(td == td.min())[0][0]
    closest_ts = dataset.time[closest_ts_idx]
    # Initialize RMSE arrays
    rmse_wet_flag = xr.DataArray(np.empty((len(A), len(B), len(C)))*np.nan,
                        coords=(A, B, C), dims=('A','B','C'))
    rmse_no_flag = xr.DataArray(np.empty((len(A), len(B), len(C)))*np.nan,
                        coords=(A, B, C), dims=('A','B','C'))
    # Create parameter space
    A = np.arange(1, 3.1, 1)
    B = np.arange(0, 1.01, 0.2)
    C = np.arange(0, 1.001, 0.05)
    # Brute-force loop
    for a in A:
        ds = calc_delta_cross_ratio(dataset, A=a, inplace=False)
        for b in B:
            ds = calc_delta_gamma(ds, B=b, inplace=False)
            for c in C:
                # print(f'A={a}; B={b}; C={c}')
                ds = clip_delta_gamma_outlier(ds)
                ds = calc_snow_index(ds)
                ds = calc_snow_index_to_snow_depth(ds, C=c, inplace=False)
                ds = id_newly_wet_snow(ds)
                ds = id_wet_negative_si(ds)
                ds = id_newly_frozen_snow(ds)
                ds = flag_wet_snow(ds)
                # Compare snow depths - mask wet snow
                mask_wet = ~(pd.isnull(ds['lidar-sd']) | pd.isnull(ds['snow_depth'].sel(time=closest_ts)) | 
                        ds['wet_snow'].sel(time=closest_ts).astype(bool))
                diff_wet = ds['lidar-sd'].where(mask_wet) - ds['snow_depth'].sel(time=closest_ts).where(mask_wet)
                rmse_wet = float(np.sqrt((diff_wet**2).sum()/len(diff_wet.values.flatten())))
                rmse_wet_flag.loc[a, b, c] = rmse_wet
                # Compare snow depths - no wet snow mask
                mask = ~(pd.isnull(ds['lidar-sd']) | pd.isnull(ds['snow_depth'].sel(time=closest_ts)))
                diff = ds['lidar-sd'].where(mask) - ds['snow_depth'].sel(time=closest_ts).where(mask)
                rmse = float(np.sqrt((diff**2).sum()/len(diff.values.flatten())))
                rmse_no_flag.loc[a,b,c] = rmse

    # After loop, save RMSE results per file
    rmse_wet_flag.to_netcdf(f'rmse_out/{ds_name}_wet_flag.nc')
    rmse_no_flag.to_netcdf(f'rmse_out/{ds_name}_no_flag.nc')
    

2023-03-03 08:46:37.163453  -- starting Frasier_2021-03-19


KeyboardInterrupt: 

In [143]:
for t in ds.time:
    print(ds['s1'].sel(band='VH', time=t).values.sum())

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [107]:
A = np.arange(1, 3.1, 1)
B = np.arange(0, 1.01, 0.2)
C = np.arange(0, 1.001, 0.05)

fs = sorted(glob('rmse_out/*.nc'))
for f in fs:
    try:
        fname = f.split('/')[1]
        data = xr.open_dataarray(f)
        a, b, c = [int(f) for f in np.where(data == data.min())]
        minn = float(data.loc[A[a], B[b], C[c]].values)
        print(fname, round(minn, 4), [A[a], B[b], C[c]])
    except:
        print('Bad:', fname)

Banner_2020-02-18_no_flag.nc 0.6316 [2.0, 1.0, 0.55]
Banner_2020-02-18_wet_flag.nc 0.1905 [1.0, 1.0, 0.65]
Banner_2021-03-15_no_flag.nc 0.5596 [2.0, 1.0, 0.30000000000000004]
Banner_2021-03-15_wet_flag.nc 0.1838 [1.0, 0.8, 0.5]
Cameron_2021-03-19_no_flag.nc 0.8028 [1.0, 1.0, 1.0]
Cameron_2021-03-19_wet_flag.nc 0.1354 [1.0, 0.2, 0.55]
Dry_Creek_2020-02-19_no_flag.nc 0.4174 [3.0, 1.0, 0.25]
Dry_Creek_2020-02-19_wet_flag.nc 0.1038 [1.0, 0.8, 0.55]
Bad: Frasier_2020-02-11_no_flag.nc
Bad: Frasier_2020-02-11_wet_flag.nc
Frasier_2021-03-19_no_flag.nc 0.4599 [1.0, 1.0, 0.65]
Frasier_2021-03-19_wet_flag.nc 0.1182 [1.0, 0.6000000000000001, 0.5]


TypeError: type numpy.ndarray doesn't define __round__ method